# SCN + FLP

Pada sesi ini kita akan menyelesaikan sebuah permasalahan rantai pasok dengan penentuan lokasi pabrik dan pusat distribusi yang akan dioperasikan. Diketahui terdapat tiga buah pabrik yang memproduksi produk X. Produk akan dikirimkan ke 12 retailer melalui 9 pusat distribusi. Berapakah total biaya yang dibutuhkan? Pabrik dan pusat distribusi mana saja yang akan digunakan apabila diketahui biaya tetap (sewa/pembangunan)? Data dan permasalahan ini saya dapatkan dari kursus udemy **Supply Chain Design and Planning with Excel & Python - Haytham Omar**

**Ilustrasi:**

<img src="https://user-images.githubusercontent.com/61647791/145341565-335ce5f6-8e60-43e1-be22-ba85e1de5e22.png" width=400 height=400/>

### 1. Mengimpor Library yang Digunakan

In [1]:
#import library
from pulp import *
import pandas as pd
import numpy as np

### 2. Membuka File Excel

In [2]:
#load data
inbound = pd.read_excel("extended_model.xlsx", sheet_name="inbound", engine="openpyxl").set_index('DCS').T
outbound = pd.read_excel("extended_model.xlsx", sheet_name="outbound", engine="openpyxl").set_index('DCS')
demand = pd.read_excel("extended_model.xlsx", sheet_name="demand", engine="openpyxl").set_index('shop')
DC_costs = pd.read_excel("extended_model.xlsx", sheet_name="DC_costs", engine="openpyxl").set_index('DCS')
DC_costs["Capacity"] = 10000

plant_costs = pd.read_excel("extended_model.xlsx", sheet_name="plant+costs", engine="openpyxl").set_index('plant')

#### a. Inbound
Tabel ini berisikan biaya transportasi per produk dari pabrik menuju pusat distribusi.

In [3]:
inbound

DCS,DC_1,DC_2,DC_3,DC_4,DC_5,DC_6,DC_7,DC_8,DC_9
plant 1,22,64,72,50,32,63,37,33,80
plant 2,71,27,36,75,33,34,52,80,54
plant 3,79,26,56,56,21,65,46,42,76


#### b. Outbound
Tabel ini berisikan biaya transportasi per produk dari pusat distribusi menuju retailer.

In [4]:
outbound

,Shop 1,shop 2,Shop 3,Shop 4,Shop 5,Shop 6,Shop 7,Shop 8,Shop 9,Shop 10,Shop 11,Shop 12
DCS,,,,,,,,,,,,
DC_1,98,67,103,117,94,116,75,104,103,71,71,41
DC_2,109,85,114,50,64,53,46,62,50,98,44,43
DC_3,57,116,57,119,53,63,106,74,75,62,43,118
DC_4,113,57,92,80,56,77,55,80,92,70,67,52
DC_5,84,73,91,77,98,85,46,58,112,67,89,102
DC_6,101,69,85,92,43,96,64,73,55,111,49,46
DC_7,109,112,55,56,54,82,60,63,111,48,75,112
DC_8,118,116,56,117,55,54,71,108,93,40,115,66
DC_9,92,111,51,59,58,115,70,86,109,116,70,68


#### c. demand
Tabel demand menunjukkan jumlah permintaan unit pada 12 retailer.

In [5]:
demand

,Demand
shop,
Shop 1,881
shop 2,806
Shop 3,704
Shop 4,767
Shop 5,575
Shop 6,741
Shop 7,593
Shop 8,708
Shop 9,800


#### d. DC_costs
Tabel ini menunjukkan biaya simpan per unit serta biaya tetap pada 9 pusat distribusi.

In [6]:
DC_costs

,Variable Cost,Fixed Cost,Capacity
DCS,,,
DC_1,12,20000,10000
DC_2,15,22000,10000
DC_3,18,40000,10000
DC_4,45,50000,10000
DC_5,23,60000,10000
DC_6,44,12000,10000
DC_7,32,18000,10000
DC_8,12,70000,10000
DC_9,23,65000,10000


#### e. Plant & Costs
Tabel ini menunjukkan data ketiga pabrik meliputi harga pembuatan per unit produk, kapasitas produksi serta biaya tetap saat melakukan kegiatan produksi

In [7]:
plant_costs

,Var,capacity,fixed cost
plant,,,
plant 1,18,3000,40000
plant 2,22,5000,60000
plant 3,32,4000,50000


### 3. Menginisiasi Model

In [8]:
#inisiasi model
model = LpProblem("Extended_Model", LpMinimize)

### 4. Menambahkan Variable

<img src="https://user-images.githubusercontent.com/61647791/145673876-589ee847-d957-4ad9-b15c-201257cb8753.png" />

In [9]:
#list pabrik
plants = plant_costs.index
#list distribution center
DCs = DC_costs.index
#list retailer
shops = demand.index

#membuat list yang menghubungkan warehouse/distribution center dengan pabrik
inbound_keys = [(p,w) for p in plants for w in DCs]
#membuat list yang menghubungkan warehouse/distribution center dengan toko
outbound_keys = [(w,s) for w in DCs for s in shops]

#inbound variable
inbound_var = LpVariable.dicts('inbound', inbound_keys, 0, None, 'Integer')
#outbound variable
outbound_var = LpVariable.dicts('outbound', outbound_keys, 0, None, 'Integer')
#open DC variable
open_dc = LpVariable.dicts('open_dc', DCs, cat = 'Binary')
#open plant variable
open_plant = LpVariable.dicts('open_p', plants, cat = 'Binary')

### 5. Menambahkan Fungsi Tujuan

<img src="https://user-images.githubusercontent.com/61647791/145675129-6a5a6ed4-db46-45ef-bccb-1203fe8d17c5.png" />

In [10]:
inbound_cost = lpSum(inbound_var[p,w]*inbound.loc[p,w] for (p,w) in inbound_keys)
outbound_cost = lpSum(outbound_var[w,s]*outbound.loc[w,s] for (w,s) in outbound_keys)
production_cost = lpSum(inbound_var[p,w]*plant_costs.loc[p,"Var"] for (p,w) in inbound_keys)
opening_plant_cost = lpSum(open_plant[p]*plant_costs.loc[p,"fixed cost"] for p in plants) 
opening_DC_cost = lpSum(open_dc[w]*DC_costs.loc[w,"Fixed Cost"] for w in DCs)
DC_operating_cost = lpSum(inbound_var[p,w]*DC_costs.loc[w,"Variable Cost"] for (p,w) in inbound_keys)


model += (inbound_cost + outbound_cost + production_cost + opening_plant_cost + opening_DC_cost + DC_operating_cost)

### 6. Menambahkan Constrain
<img src="https://user-images.githubusercontent.com/61647791/145675688-3eaeb257-35ff-4f35-8293-7c5d051cceaf.png" />

In [11]:
#demand constrain
for s in shops:
    model += lpSum(outbound_var[w,s] for w in DCs) >= demand.loc[s,"Demand"]
    
#capacity constrain
for p in plants:
    model += lpSum(inbound_var[p,w] for w in DCs) <= plant_costs.loc[p,"capacity"]*open_plant[p]
    
for w in DCs:
    model += lpSum(inbound_var[p,w] for p in plants) <= DC_costs.loc[w,"Capacity"]*open_dc[w]    
    
#flow constrain
for w in DCs:
    model += lpSum(inbound_var[p,w] for p in plants) == lpSum(outbound_var[w,s] for s in shops)

In [12]:
model.solve()

1

In [13]:
value(model.objective)

1249649

In [38]:
i = []
j = []
flow = []

for p in plants:
    for w in DCs:
        if inbound_var[p,w].varValue>0:
            i.append(p)
            j.append(w)
            flow.append(inbound_var[p,w].varValue)

for w in DCs:
    for s in shops:
        if outbound_var[w,s].varValue>0:
            i.append(w)
            j.append(s)
            flow.append(outbound_var[w,s].varValue)
            
from_to = pd.DataFrame({"From":i,
                       "To":j,
                       "Flow":flow})

from_to

,From,To,Flow
0,plant 1,DC_1,3000
1,plant 2,DC_2,5000
2,plant 3,DC_2,529
3,DC_1,Shop 1,881
4,DC_1,shop 2,806
5,DC_1,Shop 3,630
6,DC_1,Shop 10,683
7,DC_2,Shop 3,74
8,DC_2,Shop 4,767
9,DC_2,Shop 5,575


### Kesimpulan:
1. Berdasarkan hasil optimasi diperoleh total biaya minimal yang dibutuhkan sebesar $1,249,649.
2. Ketiga pabrik digunakan untuk memproduksi produk X, dimana pabrik 1 memproduksi sebanyak 3000 unit, pabrik 2 sebanyak 5000 unit dan pabrik 3 sebanyak 529 unit.
3. Perusahaan menggunakan dua pusat distribusi (DC 1, dan DC 2) sebagai penghubung antara pabrik dengan retailer.
4. Apabila diilustrasikan dalam bentuk diagram, berikut adalah konfigurasi jaringan rantai pasok dari hasil optimasi:


<img src="https://user-images.githubusercontent.com/61647791/145676881-f43509f4-1b05-4b02-9965-1dce29b7bd56.png" width=400 height=400/>